In [1]:
!aws s3api get-object --request-payer "requester" --bucket visualdatanetwork --key coco/COCO_Text.zip  COCO_Text.zip
!aws s3api get-object --request-payer "requester" --bucket visualdatanetwork --key coco/captions.zip  captions.zip
!aws s3api get-object --request-payer "requester" --bucket visualdatanetwork --key coco/instances.zip  instances.zip
!aws s3api get-object --request-payer "requester" --bucket visualdatanetwork --key coco/persons.zip  persons.zip

{
    "AcceptRanges": "bytes", 
    "ContentType": "application/zip", 
    "LastModified": "Thu, 08 Jun 2017 08:23:29 GMT", 
    "ContentLength": 15039245, 
    "ETag": "\"941dce3dcb01a48b9fe6949abc788606-2\"", 
    "Metadata": {}
}
{
    "AcceptRanges": "bytes", 
    "ContentType": "application/zip", 
    "LastModified": "Thu, 08 Jun 2017 08:20:21 GMT", 
    "ContentLength": 19673183, 
    "ETag": "\"8fc6b85a781c6d7311a90de09c2a9717-3\"", 
    "Metadata": {}
}
{
    "AcceptRanges": "bytes", 
    "ContentType": "application/zip", 
    "LastModified": "Thu, 08 Jun 2017 08:19:36 GMT", 
    "ContentLength": 165168220, 
    "ETag": "\"3d7f6f4c6468925e9e1f32fec74845db-20\"", 
    "Metadata": {}
}
{
    "AcceptRanges": "bytes", 
    "ContentType": "application/zip", 
    "LastModified": "Thu, 08 Jun 2017 08:19:58 GMT", 
    "ContentLength": 73686497, 
    "ETag": "\"3fa89f3c2652cd74db588205119c1fe5-9\"", 
    "Metadata": {}
}


In [2]:
!fab kill

[localhost] local: ps auxww | grep 'celery -A dva worker * ' | awk '{print $2}' | xargs kill -9

Fatal error: local() encountered an error (return code -9) while executing 'ps auxww | grep 'celery -A dva worker * ' | awk '{print $2}' | xargs kill -9'

Aborting.

Done.


In [3]:
!unzip COCO_Text.zip
!unzip captions.zip
!unzip instances.zip
!unzip persons.zip

Archive:  COCO_Text.zip
  inflating: COCO_Text.json          
Archive:  captions.zip
  inflating: annotations/captions_train2014.json  
  inflating: annotations/captions_val2014.json  
Archive:  instances.zip
  inflating: annotations/instances_train2014.json  
  inflating: annotations/instances_val2014.json  
Archive:  persons.zip
  inflating: annotations/person_keypoints_train2014.json  
  inflating: annotations/person_keypoints_val2014.json  


In [4]:
import sys,os,random
import django
sys.path.append("../../")
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dva.settings")
django.setup()
import os, shutil, gzip, json
from django.core.files.uploadedfile import SimpleUploadedFile
from dvaapp.shared import handle_uploaded_file
from dvaapp import models
from dvaapp.models import TEvent
from dvaapp.tasks import extract_frames, export_video_by_id
from collections import defaultdict

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
train_data = json.load(file('annotations/instances_train2014.json'))
captions_train_data = json.load(file('annotations/captions_train2014.json'))
keypoints_train_data = json.load(file('annotations/person_keypoints_train2014.json'))
text_train_data = json.load(file('COCO_Text.json'))
data = defaultdict(lambda: {'image': None, 'annotations': [], 'captions': [], 'keypoints': [], 'text': []})
id_to_license = {k['id']: k for k in train_data['licenses']}
id_to_category = {k['id']: k for k in train_data['categories']}
kp_id_to_category = {k['id']: k for k in keypoints_train_data['categories']}
for entry in train_data['images']:
    entry['license'] = id_to_license[entry['license']]
    data[entry['id']]['image'] = entry
for annotation in train_data['annotations']:
    annotation['category'] = id_to_category[annotation['category_id']]
    data[annotation['image_id']]['annotations'].append(annotation)
del train_data
for annotation in captions_train_data['annotations']:
    data[annotation['image_id']]['captions'].append(annotation)
del captions_train_data
for annotation in keypoints_train_data['annotations']:
    annotation['category'] = kp_id_to_category[annotation['category_id']]
    data[annotation['image_id']]['keypoints'].append(annotation)
print annotation
for annotation in text_train_data['anns'].itervalues():
    data[annotation['image_id']]['text'].append(annotation)

{'category': {u'supercategory': u'person', u'skeleton': [[16, 14], [14, 12], [17, 15], [15, 13], [12, 13], [6, 12], [7, 13], [6, 7], [6, 8], [7, 9], [8, 10], [9, 11], [2, 3], [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7]], u'id': 1, u'keypoints': [u'nose', u'left_eye', u'right_eye', u'left_ear', u'right_ear', u'left_shoulder', u'right_shoulder', u'left_elbow', u'right_elbow', u'left_wrist', u'right_wrist', u'left_hip', u'right_hip', u'left_knee', u'right_knee', u'left_ankle', u'right_ankle'], u'name': u'person'}, u'segmentation': {u'counts': [97214, 1, 425, 4, 422, 6, 420, 9, 417, 12, 415, 13, 413, 14, 412, 15, 411, 16, 408, 19, 407, 19, 407, 19, 406, 21, 405, 21, 405, 21, 405, 21, 406, 19, 408, 18, 409, 18, 409, 16, 411, 16, 411, 16, 412, 15, 413, 13, 416, 10, 410, 6, 5, 4, 411, 8, 418, 10, 416, 12, 415, 12, 415, 12, 415, 12, 415, 12, 415, 11, 416, 11, 417, 10, 417, 10, 416, 12, 415, 12, 415, 12, 415, 12, 415, 14, 413, 15, 411, 17, 409, 19, 408, 20, 407, 21, 406, 23, 404, 25, 402, 27

In [6]:
name = "coco_train"
fname = "coco_train.zip"
f = SimpleUploadedFile(fname, "", content_type="application/zip")
v = handle_uploaded_file(f, name)
outpath = "/root/DVA/dva/media/{}/video/{}.zip".format(v.pk, v.pk)
os.system('rm  {}'.format(outpath))
command = 'aws s3api get-object --request-payer "requester" --bucket visualdatanetwork --key coco/train2014.zip  {}'.format(
        outpath)
print command
os.system(command)
extract_frames(TEvent.objects.create(video=v).pk)
video = v
buf = []
batch_count = 0

aws s3api get-object --request-payer "requester" --bucket visualdatanetwork --key coco/train2014.zip  /root/DVA/dva/media/3/video/3.zip


In [7]:
models.Region.objects.all().filter(video=v).delete()
for frame in models.Frame.objects.all().filter(video=video):
    frame_id = int(frame.name.split('_')[-1].split('.')[0])
    annotation = models.Region()
    annotation.region_type = models.Region.ANNOTATION
    annotation.video_id = v.pk
    annotation.frame_id = frame.pk
    annotation.full_frame = True
    annotation.metadata = data[frame_id]['image']
    annotation.object_name = 'metadata'
    buf.append(annotation)
    for a in data[frame_id][u'annotations']:
        annotation = models.Region()
        annotation.region_type = models.Region.ANNOTATION
        annotation.video_id = v.pk
        annotation.frame_id = frame.pk
        annotation.metadata = a
        annotation.full_frame = False
        annotation.x = a['bbox'][0]
        annotation.y = a['bbox'][1]
        annotation.w = a['bbox'][2]
        annotation.h = a['bbox'][3]
        annotation.object_name = 'coco_instance/{}/{}'.format(a[u'category'][u'supercategory'], a[u'category'][u'name'])
        buf.append(annotation)
    for a in data[frame_id][u'keypoints']:
        annotation = models.Region()
        annotation.region_type = models.Region.ANNOTATION
        annotation.video_id = v.pk
        annotation.frame_id = frame.pk
        annotation.metadata = a
        annotation.x = a['bbox'][0]
        annotation.y = a['bbox'][1]
        annotation.w = a['bbox'][2]
        annotation.h = a['bbox'][3]
        annotation.object_name = 'coco_keypoints/{}/{}'.format(a[u'category'][u'supercategory'], a[u'category'][u'name'])
        buf.append(annotation)
    for a in data[frame_id][u'text']:
        annotation = models.Region()
        annotation.region_type = models.Region.ANNOTATION
        annotation.video_id = v.pk
        annotation.frame_id = frame.pk
        annotation.text = a['utf8_string'] if 'utf8_string' in a else ""
        annotation.metadata = a
        annotation.x = a['bbox'][0]
        annotation.y = a['bbox'][1]
        annotation.w = a['bbox'][2]
        annotation.h = a['bbox'][3]
        annotation.object_name = 'text/{}/{}/{}'.format(a['class'],a['legibility'],a['language'])
        buf.append(annotation)
    for caption in data[frame_id][u'captions']:
        annotation = models.Region()
        annotation.region_type = models.Region.ANNOTATION
        annotation.video_id = v.pk
        annotation.frame_id = frame.pk
        annotation.text = caption['caption']
        annotation.full_frame = True
        annotation.object_name = 'caption'
        buf.append(annotation)
    if len(buf) > 1000:
        try:
            models.Region.objects.bulk_create(buf)
            batch_count += 1
            print "saved {}".format(batch_count)
        except:
            print "encountered an error doing one by one"
            for k in buf:
                try:
                    k.save()
                except:
                    print "skipping"
                    print k.object_name
        buf = []
try:
    models.Region.objects.bulk_create(buf)
    batch_count += 1
    print "saved {}".format(batch_count)
except:
    print "encountered an error doing one by one"
    for k in buf:
        try:
            k.save()
        except:
            print "skipping"
            print k.object_name
buf = []

saved 1
saved 2
saved 3
saved 4
saved 5
saved 6
saved 7
saved 8
saved 9
saved 10
saved 11
saved 12
saved 13
saved 14
saved 15
saved 16
saved 17
saved 18
saved 19
saved 20
saved 21
saved 22
saved 23
saved 24
saved 25
saved 26
saved 27
saved 28
saved 29
saved 30
saved 31
saved 32
saved 33
saved 34
saved 35
saved 36
saved 37
saved 38
saved 39
saved 40
saved 41
saved 42
saved 43
saved 44
saved 45
saved 46
saved 47
saved 48
saved 49
saved 50
saved 51
saved 52
saved 53
saved 54
saved 55
saved 56
saved 57
saved 58
saved 59
saved 60
saved 61
saved 62
saved 63
saved 64
saved 65
saved 66
saved 67
saved 68
saved 69
saved 70
saved 71
saved 72
saved 73
saved 74
saved 75
saved 76
saved 77
saved 78
saved 79
saved 80
saved 81
saved 82
saved 83
saved 84
saved 85
saved 86
saved 87
saved 88
saved 89
saved 90
saved 91
saved 92
saved 93
saved 94
saved 95
saved 96
saved 97
saved 98
saved 99
saved 100
saved 101
saved 102
saved 103
saved 104
saved 105
saved 106
saved 107
saved 108
saved 109
saved 110
saved 11

saved 832
saved 833
saved 834
saved 835
saved 836
saved 837
saved 838
saved 839
saved 840
saved 841
saved 842
saved 843
saved 844
saved 845
saved 846
saved 847
saved 848
saved 849
saved 850
saved 851
saved 852
saved 853
saved 854
saved 855
saved 856
saved 857
saved 858
saved 859
saved 860
saved 861
saved 862
saved 863
saved 864
saved 865
saved 866
saved 867
saved 868
saved 869
saved 870
saved 871
saved 872
saved 873
saved 874
saved 875
saved 876
saved 877
saved 878
saved 879
saved 880
saved 881
saved 882
saved 883
saved 884
saved 885
saved 886
saved 887
saved 888
saved 889
saved 890
saved 891
saved 892
saved 893
saved 894
saved 895
saved 896
saved 897
saved 898
saved 899
saved 900
saved 901
saved 902
saved 903
saved 904
saved 905
saved 906
saved 907
saved 908
saved 909
saved 910
saved 911
saved 912
saved 913
saved 914
saved 915
saved 916
saved 917
saved 918
saved 919
saved 920
saved 921
saved 922
saved 923
saved 924
saved 925
saved 926
saved 927
saved 928
saved 929
saved 930
saved 931


In [8]:
models.Frame.objects.all().filter(video=v).count()

82783

In [ ]:
export_video_by_id(TEvent.objects.create(video=v).pk)